In [30]:
import pandas as pd
import os

# Path to the directory containing chunk files
folder_path = r"C:\Users\kusha\OneDrive\Desktop\Projects\DevDolphins\Blob files\Chunks"

# List all CSV files in the folder
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# Read and concatenate all CSVs into one DataFrame
transactions_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

# Preview the result
print(transactions_df.head())


   step       customer  age gender zipcodeOri       merchant zipMerchant  \
0     0  'C1093826151'  '4'    'M'    '28007'   'M348934600'     '28007'   
1     0   'C352968107'  '2'    'M'    '28007'   'M348934600'     '28007'   
2     0  'C2054744914'  '4'    'F'    '28007'  'M1823072687'     '28007'   
3     0  'C1760612790'  '3'    'M'    '28007'   'M348934600'     '28007'   
4     0   'C757503768'  '5'    'M'    '28007'   'M348934600'     '28007'   

              category  amount  fraud  
0  'es_transportation'    4.55      0  
1  'es_transportation'   39.68      0  
2  'es_transportation'   26.89      0  
3  'es_transportation'   17.25      0  
4  'es_transportation'   35.72      0  


In [43]:
import pandas as pd
from datetime import datetime
import os

transactions_df = pd.read_csv(r"C:\Users\kusha\OneDrive\Desktop\Projects\DevDolphins\Blob files\Chunks\chunk_11_20250624_202553.csv")
customer_df = pd.read_csv(r"C:\Users\kusha\OneDrive\Desktop\Projects\DevDolphins\Blob files\customer data\customer data.csv")


In [44]:
merged_df = transactions_df.merge(
    customer_df,
    how='inner',
    left_on=['customer', 'merchant', 'category','amount'],
    right_on=['Source', 'Target', 'typeTrans','Weight']
)
merged_df

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud_x,Source,Target,Weight,typeTrans,fraud_y
0,36,'C836424975','3','M','28007','M855959430','28007','es_hyper',7.64,0,'C836424975','M855959430',7.64,'es_hyper',0
1,36,'C1392385769','3','M','28007','M348934600','28007','es_transportation',22.14,0,'C1392385769','M348934600',22.14,'es_transportation',0
2,36,'C1667002698','3','M','28007','M85975013','28007','es_food',21.40,0,'C1667002698','M85975013',21.40,'es_food',0
3,36,'C1373640509','2','M','28007','M348934600','28007','es_transportation',21.29,0,'C1373640509','M348934600',21.29,'es_transportation',0
4,36,'C784613823','5','F','28007','M348934600','28007','es_transportation',21.08,0,'C784613823','M348934600',21.08,'es_transportation',0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10133,39,'C1135495199','2','M','28007','M348934600','28007','es_transportation',3.91,0,'C1135495199','M348934600',3.91,'es_transportation',0
10134,39,'C970603000','5','M','28007','M348934600','28007','es_transportation',9.20,0,'C970603000','M348934600',9.20,'es_transportation',0
10135,39,'C845102514','4','M','28007','M348934600','28007','es_transportation',10.09,0,'C845102514','M348934600',10.09,'es_transportation',0
10136,39,'C546927133','3','M','28007','M348934600','28007','es_transportation',12.96,0,'C546927133','M348934600',12.96,'es_transportation',0


In [33]:

# Initialize the state summary
customer_merchant_summary_df = pd.DataFrame({
    "merchant": pd.Series(dtype='str'),
    "customer": pd.Series(dtype='str'),
    "txn_count": pd.Series(dtype='int'),
    "total_amount": pd.Series(dtype='float'),
    "avg_amount": pd.Series(dtype='float'),
    "total_weight": pd.Series(dtype='float'),
    "avg_weight": pd.Series(dtype='float'),
    "patid1_flag": pd.Series(dtype='bool'),
    "last_updated": pd.Series(dtype='str')
})


In [34]:
def get_detection_time():
    IST = pytz.timezone("Asia/Kolkata")
    return datetime.now(IST).strftime("%Y-%m-%d %H:%M:%S")

def detect_pattern1_using_state():
    global customer_merchant_summary_df

    detections = []
    detection_time = get_detection_time()

    # Ensure proper types
    customer_merchant_summary_df["txn_count"] = pd.to_numeric(customer_merchant_summary_df["txn_count"], errors='coerce')
    customer_merchant_summary_df["avg_weight"] = pd.to_numeric(customer_merchant_summary_df["avg_weight"], errors='coerce')
    customer_merchant_summary_df.dropna(subset=["txn_count", "avg_weight"], inplace=True)

    # Step 1: Identify merchants that crossed 50K txn threshold NOW
    merchant_total_df = customer_merchant_summary_df.groupby("merchant")["txn_count"].sum().reset_index()
    merchant_total_df.columns = ["merchant", "total_txns"]

    # Step 2: Filter to newly eligible merchants (not processed before)
    eligible_merchants_df = merchant_total_df[merchant_total_df["total_txns"] > 50000]
    newly_eligible_merchants = eligible_merchants_df["merchant"].tolist()

    if not newly_eligible_merchants:
        return pd.DataFrame()  # nothing new to check

    # Step 3: Filter state only to relevant rows
    eligible_df = customer_merchant_summary_df[
        (customer_merchant_summary_df["merchant"].isin(newly_eligible_merchants)) &
        (~customer_merchant_summary_df["patid1_flag"])  # not already flagged
    ]

    if eligible_df.empty:
        return pd.DataFrame()

    # Step 4: Compute thresholds only for eligible merchants
    weight_thresh = (
        eligible_df.groupby("merchant")["avg_weight"]
        .quantile(0.10)
        .reset_index()
        .rename(columns={"avg_weight": "weight_thresh"})
    )

    txn_thresh = (
        eligible_df.groupby("merchant")["txn_count"]
        .quantile(0.90)
        .reset_index()
        .rename(columns={"txn_count": "txn_thresh"})
    )

    # Step 5: Join thresholds
    df1 = eligible_df.merge(weight_thresh, on="merchant")
    df1 = df1.merge(txn_thresh, on="merchant")

    # Step 6: Apply filter for pattern match
    filtered = df1[
        (df1["avg_weight"] <= df1["weight_thresh"]) &
        (df1["txn_count"] >= df1["txn_thresh"])
    ]

    # Step 7: Mark detected rows and collect output
    for _, row in filtered.iterrows():
        idx = customer_merchant_summary_df[
            (customer_merchant_summary_df["merchant"] == row["merchant"]) &
            (customer_merchant_summary_df["customer"] == row["customer"])
        ].index

        customer_merchant_summary_df.loc[idx, "patid1_flag"] = True

        detections.append({
            "YStartTime": detection_time,
            "detectionTime": detection_time,
            "patternId": "PatId1",
            "ActionType": "UPGRADE",
            "customerName": row["customer"],
            "merchantId": row["merchant"]
        })

    return pd.DataFrame(detections)


In [45]:
update_customer_merchant_summary(merged_df)
pattern1_df = detect_pattern1_using_state()

if not pattern1_df.empty:
    print("Detected Pattern 1 Customers:")
    print(pattern1_df)


Detected Pattern 1 Customers:
            YStartTime        detectionTime patternId ActionType  \
0  2025-06-26 22:21:31  2025-06-26 22:21:31    PatId1    UPGRADE   
1  2025-06-26 22:21:31  2025-06-26 22:21:31    PatId1    UPGRADE   
2  2025-06-26 22:21:31  2025-06-26 22:21:31    PatId1    UPGRADE   
3  2025-06-26 22:21:31  2025-06-26 22:21:31    PatId1    UPGRADE   
4  2025-06-26 22:21:31  2025-06-26 22:21:31    PatId1    UPGRADE   

    customerName    merchantId  
0  'C1431806998'  'M348934600'  
1  'C1771717627'  'M348934600'  
2   'C229592862'  'M348934600'  
3    'C34104097'  'M348934600'  
4   'C798889135'  'M348934600'  


In [46]:
pattern1_df

,YStartTime,detectionTime,patternId,ActionType,customerName,merchantId
0,2025-06-26 22:21:31,2025-06-26 22:21:31,PatId1,UPGRADE,'C1431806998','M348934600'
1,2025-06-26 22:21:31,2025-06-26 22:21:31,PatId1,UPGRADE,'C1771717627','M348934600'
2,2025-06-26 22:21:31,2025-06-26 22:21:31,PatId1,UPGRADE,'C229592862','M348934600'
3,2025-06-26 22:21:31,2025-06-26 22:21:31,PatId1,UPGRADE,'C34104097','M348934600'
4,2025-06-26 22:21:31,2025-06-26 22:21:31,PatId1,UPGRADE,'C798889135','M348934600'


In [42]:
customer_merchant_summary_df

,merchant,customer,txn_count,total_amount,avg_amount,total_weight,avg_weight,patid1_flag,last_updated
0,'M1053599405','C1005126300',1,90.52,90.520,90.52,90.520,False,2025-06-26 22:00:42
1,'M1053599405','C100992504',2,39.76,19.880,39.76,19.880,False,2025-06-26 22:00:42
2,'M1053599405','C1010589026',1,249.49,249.490,249.49,249.490,False,2025-06-26 22:00:42
3,'M1053599405','C1014124305',1,83.48,83.480,83.48,83.480,False,2025-06-26 22:00:42
4,'M1053599405','C1017941602',2,402.33,201.165,402.33,201.165,False,2025-06-26 22:14:40
...,...,...,...,...,...,...,...,...,...
15763,'M980657600','C57136184',1,16.60,16.600,16.60,16.600,False,2025-06-26 22:14:40
15764,'M980657600','C895105444',1,500.86,500.860,500.86,500.860,False,2025-06-26 22:14:40
15765,'M980657600','C980181294',1,81.92,81.920,81.92,81.920,False,2025-06-26 22:14:40
15766,'M980657600','C993027768',1,346.65,346.650,346.65,346.650,False,2025-06-26 22:14:40
